# Daily workflow - troselab (DRAFT)

`TR TODO: 

4. Scan Curation
    1. DIFFERENT NOTEBOOK (or different Notebook section)
    2. Make intersection of folder with database
        1. list _all_ scans
        2. have all non-curated red - the others green
        3. bhave non-processed scans grey
        4. have 'CURATE LOCALLY WITH s2P' buttons behind all scans. This button should spawn a local suite2p GUI which directly loads the respective suite2p stats.npy file from the server. Folder settings for server need to be set in datajoint.json file.




CURRENT TODO

***FIX SAMESITE LOADING BUG AFTER SESSION INGEST***
1. Find better way to prevent multi-extractions with scan sets (samesite and samesessions)
2. AUX ingest and EVENTS!!!! https://github.com/SFB1089/adamacs/pull/24 https://troselab.slack.com/archives/C02GR5XE47R/p1661457553581559 
3. remove main trigger from bpod and record tracking gate. That event can then be used to match timestamps globally (simple offset)
4. check the surgery table. Maybe an optional GUI with at least virus injection would be helpful. But low prio.
5. Write a cronjob that polls the population tables every XY minutes - ideally combine with backup script so the task can run on local data that is then pushed back to tatchu.
6. SERVER: Database backup and Data folder backup
    1. Have cronjob backup database to ana2 and isilon (midnight, incremental - only add, no delete. Increment ID to be reversible?)
    2. Have cronjob backup data folders to ana2 and isilon (midnight, incremental - only add, no delete)
    3. Use second server as worker
7. Think of project splitting 


## Login

Either log in via a local config file (see [01_pipeline](./01_pipeline.ipynb)), or enter login information manually. If you are don't have your login information, contact the administrator.


In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery, event, trial, imaging, behavior
from adamacs.ingest import session as isess
from adamacs.helpers import stack_helpers as sh
from adamacs.ingest import behavior as ibe
import datajoint as dj
from rspace_client.eln import eln
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dj.__version__

[2023-08-03 14:39:35,719][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-08-03 14:39:35,765][INFO]: Connected tobiasr@172.26.128.53:3306


'0.14.1'

### RSpace connection

In [2]:
URL=dj.config['custom'].get('rspace_URL')
API_KEY=dj.config['custom'].get('rspace_API_key')
api = eln.ELNClient(URL, API_KEY)
api.get_status()

{'message': 'OK', 'rspaceVersion': '1.80.1'}

## Activation
Next, import from `adamacs.pipeline` to activate the relevant schema.

In [3]:
from adamacs.utility import *
# from adamacs.nbgui import *
from adamacs.pipeline import subject, session, surgery, scan, equipment

Assign easy names for relevant tables

In [4]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath()
    )

## Cleaning. Use with caution!

In [5]:
# subject.Subject.delete()
# session.Session.delete()
# imaging.Processing.delete()
# imaging.Curation.delete()
# event.Event.delete()
# event.BehaviorRecording.delete()

In [6]:
# (session.Session & "session_id = 'sess9FI3KWP3'").delete()
# (subject.Subject & "subject = 'ROS-1571'").delete()
# # subject.Subject.delete()
# # session.Session.delete()
# # imaging.Processing.delete()
# # imaging.Curation.delete()
# # event.Event.delete()
# # event.BehaviorRecording.delete()

In [7]:
# session.Session.drop()
# scan.Scan.drop()
# imaging.Processing.drop()
# imaging.Curation.drop()

## 1. Data upload
(not implemented currently)

## 2. Session ingest

### Define ingest GUI functions

In [16]:
# Tobias Rose 2023: Routine ingest helpers

import ipywidgets as widgets
from IPython.display import display, HTML
from natsort import natsorted, ns
import re
from tqdm.autonotebook import tqdm
from adamacs.helpers import stack_helpers as sh
from adamacs.pipeline import subject, scan, event, behavior, session, model, equipment, imaging
from element_interface.utils import find_full_path
from adamacs.paths import get_dlc_root_data_dir
import pathlib

def select_sessions(AvailableSessionDirB, do_population = False):
    
    # Personal default values
    user_defaults = get_user_defaults(AvailableSessionDirB)

    # extract lookup tables
    Project = project.fetch('project')
    Equipment = equipment.Equipment().fetch('scanner')
    Recording_Location = surgery.AnatomicalLocation().fetch('anatomical_location')
    SessionNotes = session.SessionNote.fetch('session_note')
    s2pparm = imaging.ProcessingParamSet.fetch("paramset_idx", "paramset_desc")
    session_dirs_ingested = session.SessionDirectory.fetch('session_dir')
    scan_dirs_ingested = scan.ScanPath.fetch('path')
    IngestedSessionDirA = get_session_dir_key_from_dir(scan_dirs_ingested)
    # ScanDirArrayingested = get_scan_dir_key_from_dir(scan_dirs_ingested)
    DLCModels = model.Model.fetch("model_name")
   
    
    # Define the widgets
    session_dropdowns = []
    session_checkboxes = []
    for i, session_list in enumerate(AvailableSessionDirB):
        # Dropdowns for Project, Recording Location, Equipment

        current_session = get_session_key_from_dir([session_list])
        current_subject = get_subject_key_from_dir([session_list])

        # check if directory session is already ingested - if yes: populate GUI with table values. If no: use user defaults
        query = session.Session() & f'session_id = "{current_session[0]}"'
        count = len(query.fetch('session_id'))

        # get all sessions of current animal that are already ingested
        subject_sessions = session.SessionSameSite().proj("session_id") * session.Session.proj("subject") & f'subject = "{current_subject[0]}"'
        subject_sessions_array = subject_sessions.fetch("session_id")

        if count > 0:
            # POPULATE PRESELECTION FROM DATABASE
            # get the project associated with a session
            query = session.ProjectSession() & f'session_id = "{current_session[0]}"'
            project_dropdown_value = query.fetch1("project")
            # get the location associated with a session
            query = session.Session() * scan.ScanLocation() & f'session_id = "{current_session[0]}"'
            location_dropdown_value = query.fetch("anatomical_location")
            # get the equipment associated with a session
            query = session.Session() * scan.Scan() & f'session_id = "{current_session[0]}"'
            equipment_dropdown_value = query.fetch("scanner")
            # get the note associated with a session
            query = session.SessionNote() & f'session_id = "{current_session[0]}"'
            session_note_textbox_value = query.fetch("session_note")
            if len(session_note_textbox_value) == 0:
                session_note_textbox_value = ["none"]
            # get the current SessionSameSite
            query = session.SessionSameSite() & f'session_id = "{current_session[0]}"'
            subject_session_dropdown_value = query.fetch("same_site_id")
            # check if manually curated
            key = dict(session_id = current_session)
            try:
                curated = (imaging.Curation & key).fetch1("manual_curation").astype("bool")
            except:
                curated = False
            try:
                DLCModels_dropdown_value = (model.PoseEstimation & key).fetch1("model_name")
            except:
                DLCModels_dropdown_value = [DLCModels[0]]

        else:
            # POPULATE PRESELECTION FROM USER DEFAULTS
            project_dropdown_value = Project[user_defaults[i][0]]
            location_dropdown_value = [Recording_Location[user_defaults[i][1]]]
            equipment_dropdown_value = [Equipment[user_defaults[i][2]]]
            subject_session_dropdown_value = current_session
            subject_sessions_array = current_session
            session_note_textbox_value = ["none"]
            curated = False
            DLCModels_dropdown_value =  [DLCModels[user_defaults[i][4]]]
        
        project_dropdown = widgets.Dropdown(options=Project, value=project_dropdown_value, description="Project:")
        location_dropdown = widgets.Dropdown(options=Recording_Location, value=location_dropdown_value[0], description="Location:")
        equipment_dropdown = widgets.Dropdown(options=Equipment, value=equipment_dropdown_value[0], description="Setup:")
        s2pparms_dropdown = widgets.Dropdown(options=s2pparm[1], value=s2pparm[1][user_defaults[i][3]], description="s2p parm:")
        try:
            subject_session_dropdown = widgets.Dropdown(options=subject_sessions_array, value=subject_session_dropdown_value[0], description="same site as:")
        except:
            subject_session_dropdown = widgets.Dropdown(options=current_session[0], value=current_session[0], description="same site as:")

        DLCModels_dropdown = widgets.Dropdown(options=DLCModels, value=DLCModels_dropdown_value[0], description="DLCModel:")
        
        session_note_textbox = widgets.Text(value=session_note_textbox_value[0], description='Session comment:')
       
        
        session_dropdowns.append((project_dropdown, location_dropdown, equipment_dropdown, s2pparms_dropdown, subject_session_dropdown, DLCModels_dropdown, session_note_textbox))

        
        # Checkbox for Process - check to commit for ingest and processing
        session_checkbox = widgets.Checkbox(description='run?', layout=widgets.Layout(width='auto'))
        # session_checkboxes.append(session_checkbox)

        # Checkbox for Curation - check if manually curated
        curation_checkbox = widgets.Checkbox(description='curated?', layout=widgets.Layout(width='auto'))
        session_checkboxes.append((session_checkbox, curation_checkbox))
    
    # Display the widgets
    output = widgets.Output()

    with output:
        # Display the Sessions labels and associated dropdowns and checkboxes
        hbox_list = []
        for i, session_list in enumerate(AvailableSessionDirB): #unique_directory_strings(SessionDirA, SessionDirB)
            # Create an HBox to hold the label and associated dropdowns and checkbox
            hbox = widgets.HBox()
            hbox.children = [
                widgets.Label(value=session_list + ':', layout=widgets.Layout(width='1800px')), 
                session_dropdowns[i][0],
                session_dropdowns[i][1],
                session_dropdowns[i][2],
                session_dropdowns[i][3],
                session_dropdowns[i][4],
                session_dropdowns[i][5],
                session_dropdowns[i][6],
                session_checkboxes[i][0],
                session_checkboxes[i][1]
            ]
            if session_list in unique_directory_strings(IngestedSessionDirA, AvailableSessionDirB):
                hbox.children[0].value = '*' + session_list + ':'
                hbox.children[8].value = True
                hbox.children[9].value = curated
            else:
                hbox.children[8].value = False
                hbox.children[9].value = False
                
                hbox.children[1].disabled = False
                hbox.children[2].disabled = False
                hbox.children[3].disabled = False
                hbox.children[4].disabled = False
                hbox.children[5].disabled = False
                hbox.children[6].disabled = False
                hbox.children[7].disabled = False
                
            hbox_list.append(hbox)

        vbox = widgets.VBox(hbox_list, layout=widgets.Layout(flex='0 0 auto', overflow_y='scroll'))
        # Display the commit button
        commit_button = widgets.Button(description='Commit', layout=widgets.Layout(width='auto'))
        display(vbox, commit_button)

        # Define the callback function for the commit button
        def commit_button_clicked(b):
            selected_sessions = [AvailableSessionDirB[i] for i in range(len(AvailableSessionDirB)) if session_checkboxes[i][0].value]
            selected_projects = [session_dropdowns[i][0].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i][0].value]
            selected_locations = [session_dropdowns[i][1].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i][0].value]
            selected_equipment = [session_dropdowns[i][2].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i][0].value]
            selected_s2pparms = [session_dropdowns[i][3].index for i in range(len(AvailableSessionDirB)) if session_checkboxes[i][0].value]
            selected_same_site = [session_dropdowns[i][4].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i][0].value]
            selected_DLCmodel = [session_dropdowns[i][5].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i][0].value]
            entered_session_note = [session_dropdowns[i][6].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i][0].value]

            selected_scans = get_scan_key_from_dir(selected_sessions)
            selected_sessions = get_session_key_from_dir(selected_sessions)

            selected_s2pparms_index = s2pparm[0][selected_s2pparms]

            output.clear_output()

            
            with output:
                # OUTPUT FUNCTION: PERFORM INGESTIONS AND JOBS BASED ON SELECTION

                # Ingest selected sessions here
                populate_settings = {'display_progress': True, 'suppress_errors': True, 'processes': 1}

                ## SESSION processing
                # ToDo: Make a separate CURATION PROCESSING loop - and Exclude CURATION-CHECKED AND RUNCHECKED HERE. 
                # If BOTH run and curation are clicked, generate a LOAD processing task, a manual curation task and corresponding populations
                for i, sessi in enumerate(tqdm(selected_sessions, desc='Current Session')):
                    isess.ingest_session_scan(sessi, verbose=False, project_key=selected_projects[i], equipment_key=selected_equipment[i], location_key=selected_locations[i], software_key='ScanImage')
                    # update / insert session info based on user choice from above
                    try:
                        session.SessionNote.insert1({'session_id': sessi, 'session_note': entered_session_note[i]})
                    except:
                        session.SessionNote.delete_quick({'session_id': sessi})
                        session.SessionNote.insert1({'session_id': sessi, 'session_note': entered_session_note[i]})
                    
                    session.SessionSameSite.update1({'session_id': sessi, 'same_site_id': selected_same_site[i]})
                    
                    # update scaninfo based on user choice from above
                   
                    ## SCAN Processing 
                    # get the scans associated with a session
                    query = session.Session() * scan.Scan() & f'session_id = "{sessi}"'
                    scans_to_process = query.fetch("scan_id")
                    
                    for j, scansi in enumerate(scans_to_process):
                        try:
                            scan.Scan.update1({'session_id': sessi, 'scan_id': scansi, 'scan_notes': entered_session_note[i]})
                        except:
                            scan.Scan.insert1({'session_id': sessi, 'scan_id': scansi, 'scan_notes': entered_session_note[i]})
                        try:
                            scan.ScanLocation.update1({'session_id': sessi, 'scan_id': scansi, 'anatomical_location': selected_locations[i]})
                        except: 
                            scan.ScanLocation.insert1({'session_id': sessi, 'scan_id': scansi, 'anatomical_location': selected_locations[i]})
                        
                        query = scan.ScanPath() & 'scan_id = "' + scansi + '"'
                        dir_proc = query.fetch('path')[0]

                        # push scan to ProcessingTask
                        # TODO: handle multiscan concatenation from here?
                        imaging.ProcessingTask.insert1((sessi, scansi, selected_s2pparms_index[i], dir_proc, 'trigger'), skip_duplicates=True)

                            
                    scan.ScanInfo.populate(**populate_settings) 
                    
                    ## AUX Processing
                    for j, scansi in enumerate(scans_to_process):
                                            # get the equipment/experiment class from userfunction_consolidate_files associated with a scan

                        print('- - - -')
                        print('Ingesting AUX, STIM, and BPOD data for scan:', scansi)
                        
                        aux_setup_typestr = (scan.ScanInfo() & 'scan_id = "' + scansi + '"').fetch("userfunction_info")[0]    
                        
                        try:
                            ibe.ingest_aux(sessi,scansi,verbose=False, aux_setup_type=aux_setup_typestr) # ingests AUX into event.Event and event.BehaviorRecording
                        except:
                            print(f'Failed to ingest aux for scan = "{scansi}"')
                            
                        try:
                            
                            if aux_setup_typestr == "bench2p":
                                # print('bench2p - detected stimuli and trials:')
                                stimuli = (event.Event &  'scan_id = "' + scansi + '"' & 'event_type LIKE "%;%"').fetch("event_type")
                                trials = len(set(stimuli))
                                trial_stims = {x: list(stimuli).count(x) for x in stimuli}
                                # print(trial_stims)
                                
                                ibe.get_and_ingest_trial_times(scansi, aux_setup_typestr) 

                                # print('bench2p - Ingesting running wheel data')
                                
                            if aux_setup_typestr == "openfield":
                                # print('openfield - detected stimuli and trials:')
                                stimuli = (event.Event &  'scan_id = "' + scansi + '"' & 'event_type LIKE "%;%"').fetch("event_type")
                                trials = len(set(stimuli))
                                trial_stims = {x: list(stimuli).count(x) for x in stimuli}
                                # print(trial_stims)
                                
                                ibe.get_and_ingest_trial_times(scansi, aux_setup_typestr) 

                                # print('openfield - Ingesting IMU data')
                        except:
                            print(f'Failed to ingest stim data for scan = "{scansi}"')
                            
                        try:
                            ibe.ingest_bpod(sessi,scansi,verbose=False, aux_setup_type=aux_setup_typestr)
                        except:
                            print(f'Failed to ingest bpod data for scan = "{scansi}"')        
                    
                    ## Task processing and population
                    try:
                        if aux_setup_typestr == "bench2p":
                            behavior.TreadmillRecording.populate()
                        if aux_setup_typestr == "openfield":
                            behavior.HarpRecording.populate()
                    except:
                        print(f'Failed to populate HARP/Treadmill data for scan = "{scansi}"')  
                        
                    
                    ## DLC processing and population
                    try:
                        for j, scansi in enumerate(scans_to_process):
                            aux_setup_typestr = (scan.ScanInfo() & 'scan_id = "' + scansi + '"').fetch("userfunction_info")[0]

                            print('- - - -')
                            print('DLC pose estimation:', scansi)
                            
                            # insert TOP movie into model table
                            scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0] 
                            moviepath = str(list(pathlib.Path((scan.ScanPath() & scan_key).fetch("path")[0]).glob("*top*.mp4*"))[0])
                            
                            key = {'session_id': scan_key["session_id"],
                                'recording_id': scan_key["scan_id"], 
                                'camera': "mini2p1_top", # Currently 'scanner' due to in equipment tables
                                }
                            model.VideoRecording.insert1(key, skip_duplicates=True)
                            key.update({'file_path': moviepath,
                                        'file_id': 0})  #INCREMENT FILE_ID WITH CAM NUMBER?
                            
                            model.VideoRecording.File.insert1(key, ignore_extra_fields=True, skip_duplicates=True)
                            
                            key =  (model.VideoRecording & f'recording_id="{scansi}"').fetch1('KEY')
                            key.update({'model_name': selected_DLCmodel[i], 'task_mode': 'trigger'})        
                            
                            model.PoseEstimationTask.insert_estimation_task(key, key["model_name"], analyze_videos_params={'save_as_csv':True, 'dynamic':(True,.5,60)}) # dynamic cropping
                        
                        ## DLC population
                        model.RecordingInfo.populate()
                        model.PoseEstimation.populate()
                    except:
                        print(f'Failed to populate DLC data for scan = "{scansi}"')  
                    
                    try:        
                        ## Make RSpace entries 
                        query = session.Session() * subject.User() & f'session_id = "{sessi}"'
                        animalID = query.fetch("subject")[0]
                        date = query.fetch("session_datetime")[0].strftime("%Y-%m-%d")
                        userID = query.fetch("initials")[0]
                        sessionID = query.fetch("session_id")[0]
                        fetchtable = session.Session() * scan.ScanPath() * session.SessionUser() * session.ProjectSession() * session.SessionNote() * session.SessionSameSite() * scan.Scan() *scan.ScanInfo() & f'session_id = "{sessi}"'
                        # include stimulus / task info
                        
                        make_rspace_session_document(animalID, sessi, date, userID, fetchtable)
                    except:
                        print(f'Failed to upload Rspace data for session = "{sessi}"')  
                
                    
                    
                # print('Ingested Scans:', scans_to_process)



                # RUN PRIMARY PROCESSING TASKS (S2P)
                
                if do_population:
                    
                    print("\n---- Populate imported and computed imaging tables ----")
                    
                    try:
                        imaging.Processing.populate(**populate_settings)

                        try:
                            for i, sessi in enumerate(set(selected_sessions)):
                                # get the scans associated with a session
                                query = session.Session() * scan.Scan() & f'session_id = "{sessi}"'
                                scans_to_process = query.fetch("scan_id")
                                for j, scansi in enumerate(scans_to_process):
                                    imaging.Curation().create1_from_processing_task({'session_id': sessi, 'scan_id': scansi, "paramset_idx": selected_s2pparms_index[i], "manual_curation": session_checkboxes[i][1].value})
                        except:
                            print(f'No curation entries found for {sessi}')                      

                        imaging.MotionCorrection.populate(**populate_settings)

                        imaging.Segmentation.populate(**populate_settings)

                        imaging.MaskClassification.populate(**populate_settings)

                        imaging.Fluorescence.populate(**populate_settings)

                        imaging.Activity.populate(**populate_settings)

                        print("\n---- Successfully completed workflow ----")

                        ## Append RSpace Template figures
                        # TODO: make movies and upload as well 
                        for i, sessi in enumerate(set(selected_sessions)):
                            # get the scans associated with a session
                            query = session.Session() * scan.Scan() & f'session_id = "{sessi}"'
                            scans_to_process = query.fetch("scan_id")
                            for j, scansi in enumerate(scans_to_process):
                                session_key = (session.Session & f'session_id = "{sessi}"').fetch('KEY')[0]
                                scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]
                                curation_key = (imaging.Curation & scan_key & 'curation_id=1').fetch1('KEY')
                                print(curation_key)
                                rspace_id = (session.SessionRspace & f'session_id = "{sessi}"').fetch('rspace_id')[0]
                        
                                make_overview_figures_rspace(curation_key, rspace_id)
                                
                                print('making movie')
                                make_overview_movies_rspace(curation_key, rspace_id)
                    except: 
                        print(f'IMAGING POPULATION DID NOT RUN FOR {sessi}')
    # Attach the callback function to the commit button
    commit_button.on_click(commit_button_clicked)

    # Display the output
    display(output)
        




def make_rspace_session_document(animalID, sessionID, date, userID, fetchtable):
    # find 'Experiments' folder and get ID
    folders = api.list_folder_tree()
    experiments_ids = []
    for record in folders['records']:
        if record['name'] == 'Experiments':
            experiments_ids.append(record['id'])

    # find folders under 'Experiments' folder
    names_and_ids = []
    subfolders = api.list_folder_tree(experiments_ids[0])
    names_and_ids.append([(record['name'], record['id']) for record in subfolders['records']])

    # find matches with animal IDs
    match_found = any(animalID in item[0] for sublist in names_and_ids for item in sublist)

    # if found, find folder id matching that animal
    if match_found:
        save_folder_id = [x[1] for sublist in names_and_ids for x in sublist if animalID in x[0]][0]
    else:
        # if not generate the directory
        new_folder = api.create_folder(name=f"{userID}_{animalID}", parent_folder_id=experiments_ids[0], notebook=True)
        save_folder_id = new_folder['id']


    # find matches with document IDs

    # find all documents in the animal folder
    all_animal_documents = api.list_folder_tree(save_folder_id)
    doc_names_and_ids = []
    doc_names_and_ids.append([(record['name'], record['id']) for record in all_animal_documents['records']])
    # find matches with sessionID
    match_found = any(sessionID in item[0] for sublist in doc_names_and_ids for item in sublist)

    if match_found:
        ids = [id for string, id in doc_names_and_ids[0] if sessionID in string]
        new_doc = {"id": ids[0]}
    else:
        new_doc = api.create_document(name=f"{date}_{sessionID}", parent_folder_id=save_folder_id)

    # create RSpace document
    df = fetchtable.fetch(format='frame')
    html_table = df.to_html()
    content = html_table
    api.append_content(new_doc['id'], content)

    # ingest document in rspace table
    session.SessionRspace.insert1({'session_id': sessionID, 'rspace_id': new_doc['id']}, skip_duplicates=True)
    # ingest animal folder id in subject Rspace table
    subject.SubjectRspace.insert1({'subject': animalID, 'rspace_subject_id': save_folder_id}, skip_duplicates=True)
    # session.SessionRspace.insert1({'rspace_id': new_doc['id'], 'rspace_url': new_doc['_links'][0]['link']})



def get_session_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]
     
def get_scan_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]

def get_session_key_from_dir(string):
    result = [re.search(r'sess\S+', item).group(0) for item in string]
    return result

def get_user_initials_from_dir(string):
    result = [name[:2] for name in string]
    return result

def get_subject_key_from_dir(string):
    result = [item.split("_")[1] for item in string]
    return result

def get_date_key_from_dir(directory):
    return directory.split("_")[-1]

def get_scan_key_from_dir(string):
    result = [re.search(r'scan\S+_', item).group(0)[:-1] for item in string]
    return result

def unique_directory_strings(dirs1, dirs2):
    set1 = set(dirs1)
    set2 = set(dirs2)
    common_dirs = list(set1.intersection(set2))
    unique_dirs = list(set(set1.union(set2)) - set(common_dirs))
    return unique_dirs

def get_user_defaults(directory):
    RN = [2, 2, 0, 0, 0]
    JJ = [4, 8, 7, 0, 0]
    TR = [2, 2, 0, 0, 0]
    LK = [7, 5, 0, 0, 0]
    DB = [4, 0, 1, 0, 0]
    NK = [6, 5, 7, 0, 0]
    LE = [8, 5, 0, 3, 0]
    
    user_initials = get_user_initials_from_dir(directory)

    user_arrays = {}
    for initial in user_initials:
        values = locals()[initial]
        user_arrays[initial] = values

    new_array = [user_arrays[initial] for initial in user_initials]
    return new_array

def make_overview_figures_rspace(curation_key, rspace_id):
    # Figure Style settings for notebook.

    import matplotlib as mpl
    mpl.rcParams.update({
        'axes.spines.left': False,
        'axes.spines.bottom': False,
        'axes.spines.top': False,
        'axes.spines.right': False,
        'legend.frameon': False,
        'figure.subplot.wspace': .01,
        'figure.subplot.hspace': .01,
        'figure.figsize': (18, 13),
        'ytick.major.left': False,
        'xtick.major.bottom': False
    })
    jet = mpl.cm.get_cmap('jet')
    jet.set_bad(color='k')

    ref_image = (imaging.MotionCorrection.Summary & curation_key & 'field_idx=0').fetch1('ref_image')
    average_image = (imaging.MotionCorrection.Summary & curation_key & 'field_idx=0').fetch1('average_image')
    correlation_image = (imaging.MotionCorrection.Summary & curation_key & 'field_idx=0').fetch1('correlation_image')
    max_proj_image = (imaging.MotionCorrection.Summary & curation_key & 'field_idx=0').fetch1('max_proj_image')
    
    plt.ioff()
    plt.subplot(1, 4, 1)
    plt.imshow(ref_image, cmap='gray', )
    plt.title("Reference Image for Registration");

    plt.subplot(1, 4, 2)
    plt.imshow(average_image, cmap='gray')
    plt.title("Registered Image, Mean Projection");

    plt.subplot(1, 4, 3)
    plt.imshow(max_proj_image, cmap='gray')
    plt.title("Registered Image, Max Projection")

    plt.subplot(1, 4, 4)
    plt.imshow(correlation_image, cmap='gray')
    plt.title("Registered Image, Correlation Map")

    tmpdir = dj.config['custom'].get('suite2p_fast_tmp')[0]
    session_id = curation_key['session_id']
    scan_id = curation_key['scan_id']
    save_path = os.path.join(tmpdir, f'{session_id}_{scan_id}_templates.png')
    plt.savefig(save_path)
    plt.clf()

    # print(f'rspace_id' {rspace_id})
    # print(f'save_path' {save_path})
    # print(f'session_id' {session_id})
    
    append_rspace_session_image(rspace_id, save_path, captionimage='Suite2p Templates ' + session_id)   
    plt.ion()

    
def make_overview_movies_rspace(curation_key, rspace_id):
    # params_key = (imaging.ProcessingParamSet & 'paramset_idx = "4"').fetch('KEY')
    # reg_tiffs_available = (imaging.ProcessingParamSet & params_key).fetch("params")[0]['reg_tif']
    from scipy.ndimage import mean
    import tifffile
    path = (scan.ScanPath & curation_key).fetch1("path") + ("/suite2p/plane0/reg_tif")

    # path = '/datajoint-data/data/jisooj/RN_OPI-1681_2023-02-15_scan9FGLEFJ3_sess9FGLEFJ3/suite2p_exp9FGLEFJ3/suite2p/plane0/reg_tif'
    # Get a list of all tiff files in the folder
    tiff_files = [os.path.join(path, f) for f in natsorted(os.listdir(path)) if f.endswith('.tif')]

    # print(tiff_files)

    # Load each tiff stack into a list of numpy arrays
    stacks = []
    for f in tiff_files:
        with tifffile.TiffFile(f) as tif:
            # Get the number of pages in the file
            num_pages = len(tif.pages)
            
            # Create a numpy array to store all pages
            stack = np.zeros((num_pages,) + tif.pages[0].shape, dtype=tif.pages[0].dtype)
            
            # Iterate over the pages and store them in the array
            for i, page in enumerate(tif.pages):
                stack[i] = page.asarray()

        stacks.append(stack)

    # Concatenate the stacks into a single numpy array along the z-axis
    volume = np.concatenate(stacks, axis=0)

    # delete registration tiff
    for f in tiff_files:
        os.remove(f) 
    
    ### moving average filter
    # Create a running Z mean projection of the volume

    runav = 20
    # running_z_projection = uniform_filter_mt(volume, size=(runav,xyrunav,xyrunav))
    running_z_projection = sh.rolling_average_filter(volume, runav)

    session_id = curation_key['session_id']
    scan_id = curation_key['scan_id']

    filename = os.path.join(path, 'registered_movie_' + session_id + '_' + scan_id + '_' + str(runav) + '_frame_runningaverage2' + '.mp4')

    fps = 120   # frames per second - 120 default
    p1 = 2       # percentile scaling low - 1 default
    p2 = 99.998  # percentile scaling high - 99.995 default

    rescaled_image_8bit = sh.make_stack_movie(running_z_projection, filename, fps, p1, p2)

    tmpdir = dj.config['custom'].get('suite2p_fast_tmp')[0]


    ### movie uload does not work yet
    # append_rspace_session_image(rspace_id, filename, captionimage='registered movie ' + scan_id)   

def append_rspace_session_image(rspace_id, filename, captionimage='no caption'):
    with open(filename, 'rb') as f:
        uploaded_image = api.upload_file(f, caption=captionimage)
        
        content = f"""
        <p>Suite2p templates of dataset {filename}
        <p>
        <fileId={uploaded_image['id']}>
        """
        api.append_content(rspace_id, content)
        
        print(f"uploaded image id = {uploaded_image['id']}") 



#### Parse user data root dir and sort by date-string in directory name

### Ingest GUI

Please follow these instructions to add new recording sessions using the provided GUI:

**Step 1: Add new recording sessions**

Use the GUI to add new recording sessions and link them to projects, brain regions, suite2p options, and previous sessions of the same animal that you have imaged repeatedly.

The GUI does the following:

- it parses your primary data storage folder on the analysis server and lists all session directories
- it marks all *new* folders with an asterisk and a pre-checked "run" checkbox
- all session folders that are already in the database are unchecked - but the metadata can still be modified and committed when you check the checkbox

What you need to do:

1. Upload your folders to the server under your own credentials and folder name
2. In your `dj.config` file you can store the default values for the selectable metadata (not fully implemented)
3. Edit the selectable metadata, select the appropriate s2p config file and select the appropriate first session of a chronic experiment with that animal. Finally: Add some comments if needed. 
4. To queue for processing check checkbox 
5. Press **commit**

This will 
- pull all animal information from the pyrat database to populate the `subject` tables
- ingest the session, generate a new `session` table entry, pull and store all metadata from the scanimage tiff to populate the `scan` tables
- put all necessary data into the processing queue for image registration, source extraction etc. Currently this is triggered immediately - in the future our two servers will poll the processing queue regularly to work in parallel on the data

Importantly, the ingest will generate (or write into if existing) the `animal_ID` folder of **RSPACE**. It will generate a session document and post all important session information right there. If available it will also automatically upload the scanimage template images. Feel free to add information (bloodvessel overviews, session comments, intermediate analyses etc.) to this rspace session document. 

Furthermore, this will
- read in all 'h5' wavesurfer files to populate the `event` tables

**Step 2: Manual Curation**

If you have manually curated the data, tick the manual curation checkbox. This will trigger another source extraction based on that curation. You can add as many s2p settings to the tables as you wish.

## ToDo
1. integrate BPOD ingestion (needs new example file! Bpod data needs to include Main gate as events!)
2. integrate HARP ingestion (check consistence of frame numbers to be expected from cam rect and 2p rec)
3. 


In [17]:
imaging.Curation()

session_id,scan_id,paramset_idx,curation_id,curation_time time of generation of this set of curated results,"curation_output_dir output directory of the curated results, relative to root data directory",manual_curation has manual curation been performed on this result?,curation_note
sess9FGLE1FN,scan9FGLE1FN,3,1,2023-08-02 12:29:02,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-15_scan9FGLE1FN_sess9FGLE1FN,0,
sess9FGLEFJ3,scan9FGLEFJ3,0,1,2023-08-02 12:40:10,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-15_scan9FGLEFJ3_sess9FGLEFJ3,0,
sess9FGLEFJ3,scan9FGLEFJ3,3,1,2023-08-02 09:59:38,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-15_scan9FGLEFJ3_sess9FGLEFJ3,0,
sess9FGLEFJ3,scan9FGLEFJ3,3,2,2023-08-02 12:40:10,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-15_scan9FGLEFJ3_sess9FGLEFJ3,0,
sess9FH37WBZ,scan9FH37WBZ,0,1,2023-08-01 12:32:53,/datajoint-data/data/tobiasr/JJ_ROS-1487_2023-03-17_scan9FH37WBZ_sess9FH37WBZ,0,
sess9FHDA7AI,scan9FHDA7AI,0,1,2023-08-02 11:56:57,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-03_scan9FHDA7AI_sess9FHDA7AI,0,
sess9FHDA7AI,scan9FHDADB9,0,1,2023-08-02 11:57:53,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-03_scan9FHDADB9_sess9FHDA7AI,0,
sess9FHELAYA,scan9FHELAYA,0,1,2023-08-01 12:52:12,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELAYA_sess9FHELAYA,0,
sess9FHELAYA,scan9FHELMRY,0,1,2023-08-01 12:53:27,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELMRY_sess9FHELAYA,0,
sess9FHELAYA,scan9FHELX2S,0,1,2023-08-01 12:54:55,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELX2S_sess9FHELAYA,0,


In [9]:
# get content of user directory
import fnmatch
dataroot = dj.config['custom']['exp_root_data_dir'][0]
# dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and '_' in d]
dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and fnmatch.fnmatch(d, 'NK*') and fnmatch.fnmatch(d, '*9FI*')]
sorted_dirs_root = natsorted(dirs_root, key=get_date_key_from_dir, reverse = True)
sorted_dirs_root

['NK_ROS-1576_2023-07-10_scan9FIZNWNS_sess9FIZNWNS',
 'NK_WEZ-8869_2023-06-07_scan9FIG3MU8_sess9FIG39RU',
 'NK_WEZ-8869_2023-06-07_scan9FIG3GCJ_sess9FIG39RU',
 'NK_WEZ-8869_2023-06-07_scan9FIG39RU_sess9FIG39RU',
 'NK_WEZ-8869_2023-05-31_scan9FIBUH7F_sess9FIBUH7F',
 'NK_WEZ-8869_2023-05-31_scan9FIBTYBC_sess9FIBTYBC',
 'NK_WEZ-8869_2023-05-31_scan9FIBTDD0_sess9FIBTDD0',
 'NK_WEZ-8869_2023-05-31_scan9FIBT7CK_sess9FIBT7CK']

In [10]:
# Generate GUI
print("\033[1m" + 'INGEST GUI' + "\033[0m" )
selected_data = select_sessions(sorted_dirs_root, do_population = True)

INGEST GUI


Output()

In [18]:
scansi = "scan9FIG3GCJ"
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')[0]
curation_key = (imaging.Curation & scan_key & 'curation_id=1').fetch1('KEY')

In [25]:
imaging.Curation.insert1(
    dict(
        **scan_key,
        paramset_idx=0,
        curation_id=10,
        curation_time="2021-04-30 12:22:15.032",
        curation_output_dir="/datajoint-data/data/tobiasr/NK_WEZ-8869_2023-06-07_scan9FIG3GCJ_sess9FIG39RU",
        manual_curation=True,
        curation_note="",
    )
)

In [34]:
scan.Scan & scan_key

session_id,scan_id,scanner,acq_software,scan_notes free-notes
sess9FIG39RU,scan9FIG3GCJ,mini2p_05,ScanImage,BonnBrain


In [30]:
imaging.MotionCorrection.populate(display_progress=True)
imaging.Segmentation.populate(display_progress=True)
imaging.Fluorescence.populate(display_progress=True)
imaging.Activity.populate(display_progress=True)
# imaging_report.ScanLevelReport.populate(display_progress=True)
# imaging_report.TraceReport.populate(display_progress=True)

Activity: 100%|██████████| 1/1 [00:00<00:00, 16.00it/s]


NameError: name 'imaging_report' is not defined

In [32]:
imaging.Fluorescence()

session_id,scan_id,paramset_idx,curation_id
sess9FGLE1FN,scan9FGLE1FN,3,1
sess9FGLEFJ3,scan9FGLEFJ3,0,1
sess9FGLEFJ3,scan9FGLEFJ3,3,1
sess9FGLEFJ3,scan9FGLEFJ3,3,2
sess9FH37WBZ,scan9FH37WBZ,0,1
sess9FHDA7AI,scan9FHDA7AI,0,1
sess9FHDA7AI,scan9FHDADB9,0,1
sess9FHELAYA,scan9FHELAYA,0,1
sess9FHELAYA,scan9FHELMRY,0,1
sess9FHELAYA,scan9FHELX2S,0,1


In [12]:
model.VideoRecording() * model.VideoRecording.File() * equipment.Device() * session.SessionUser() * subject.User()

session_id,recording_id,file_id,camera,user_id,"file_path filepath of video, relative to root data directory",camera_description,name,shorthand TR: added name_FirstInitialSurname shorthand,initials Update after pyrat ingestion,email TR: for completeness' sake also email,"lab short lab name, pyrat labid"
sess9FHDA7AI,scan9FHDA7AI,0,mini2p1_top,7,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-03_scan9FHDA7AI_sess9FHDA7AI/scan9FHDA7AI_top_video_2023-03-31T16_04_25.mp4,"Basler a2A1920-160umBAS, Xx objective",Narayanamurthy Rukhmani,rukhun,RN,rnar@uni-bonn.de,Rose
sess9FHDA7AI,scan9FHDADB9,0,mini2p1_top,7,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-03_scan9FHDADB9_sess9FHDA7AI/scan9FHDADB9_top_video_2023-03-31T16_04_25.mp4,"Basler a2A1920-160umBAS, Xx objective",Narayanamurthy Rukhmani,rukhun,RN,rnar@uni-bonn.de,Rose
sess9FHELAYA,scan9FHELAYA,0,mini2p1_top,7,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELAYA_sess9FHELAYA/scan9FHELAYA_top_video_2023-04-05T15_19_53.mp4,"Basler a2A1920-160umBAS, Xx objective",Narayanamurthy Rukhmani,rukhun,RN,rnar@uni-bonn.de,Rose
sess9FHELAYA,scan9FHELMRY,0,mini2p1_top,7,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELMRY_sess9FHELAYA/scan9FHELMRY_top_video_2023-04-05T15_41_39.mp4,"Basler a2A1920-160umBAS, Xx objective",Narayanamurthy Rukhmani,rukhun,RN,rnar@uni-bonn.de,Rose
sess9FHELAYA,scan9FHELX2S,0,mini2p1_top,7,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHELX2S_sess9FHELAYA/scan9FHELX2S_top_video_2023-04-05T16_01_14.mp4,"Basler a2A1920-160umBAS, Xx objective",Narayanamurthy Rukhmani,rukhun,RN,rnar@uni-bonn.de,Rose
sess9FHELAYA,scan9FHEMBE2,0,mini2p1_top,7,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-04-05_scan9FHEMBE2_sess9FHELAYA/scan9FHEMBE2_top_video_2023-04-05T16_27_55.mp4,"Basler a2A1920-160umBAS, Xx objective",Narayanamurthy Rukhmani,rukhun,RN,rnar@uni-bonn.de,Rose
sess9FHS7Y22,scan9FHS7Y22,0,mini2p1_top,3,/datajoint-data/data/tobiasr/NK_ROS-1485_2023-04-28_scan9FHS7Y22_sess9FHS7Y22/scan9FHS7Y22_top_video_2023-04-28T12_42_12.mp4,"Basler a2A1920-160umBAS, Xx objective",Krasilshchikova Natalia,nataliak,NK,nkra1@uni-bonn.de,Rose
sess9FHS7Y22,scan9FHS845A,0,mini2p1_top,3,/datajoint-data/data/tobiasr/NK_ROS-1485_2023-04-28_scan9FHS845A_sess9FHS7Y22/scan9FHS845A_top_video_2023-04-28T12_53_47.mp4,"Basler a2A1920-160umBAS, Xx objective",Krasilshchikova Natalia,nataliak,NK,nkra1@uni-bonn.de,Rose
sess9FI3KWP3,scan9FI3KWP3,0,mini2p1_top,7,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-05-17_scan9FI3KWP3_sess9FI3KWP3/scan9FI3KWP3_top_video_2023-05-17T14_38_34.mp4,"Basler a2A1920-160umBAS, Xx objective",Narayanamurthy Rukhmani,rukhun,RN,rnar@uni-bonn.de,Rose
sess9FI3KWP3,scan9FI3L5PZ,0,mini2p1_top,7,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-05-17_scan9FI3L5PZ_sess9FI3KWP3/scan9FI3L5PZ_top_video_2023-05-17T14_55_15.mp4,"Basler a2A1920-160umBAS, Xx objective",Narayanamurthy Rukhmani,rukhun,RN,rnar@uni-bonn.de,Rose


In [15]:
(imaging.Curation()).fetch(order_by = "curation_time", format = "frame")

curation_time   
session_id   scan_id      paramset_idx curation_id                       
sess9FJBPNSP scan9FJBPNSP 0            1           2023-08-01 12:08:54  \
                                       2           2023-08-01 12:08:54   
                                       3           2023-08-01 12:08:54   
sess9FH37WBZ scan9FH37WBZ 0            1           2023-08-01 12:32:53   
sess9FHELAYA scan9FHELAYA 0            1           2023-08-01 12:52:12   
             scan9FHELMRY 0            1           2023-08-01 12:53:27   
             scan9FHELX2S 0            1           2023-08-01 12:54:55   
             scan9FHEMBE2 0            1           2023-08-01 12:56:24   
sess9FIG39RU scan9FIG39RU 0            1           2023-08-01 13:07:36   
                                       2           2023-08-01 13:07:36   
             scan9FIG3GCJ 0            1           2023-08-01 13:08:18   
                                       2           2023-08-01 13:08:18   
             scan9FIG3MU8 0            1           2023-08-01 13:09:30   
                                       2           2023-08-01 13:09:30   
sess9FJ4IJOQ scan9FJ4IJOQ 3            1           2023-08-01 13:47:12   
sess9FJ5KHBX scan9FJ5KHBX 3            1           2023-08-01 13:51:04   
sess9FJ5L18B scan9FJ5L18B 3            1           2023-08-01 14:13:00   
sess9FJ5L9VD scan9FJ5L9VD 3            1           2023-08-01 14:22:07   
sess9FJ5LHYE scan9FJ5LHYE 3            1           2023-08-01 14:31:36   
sess9FJ5LVCL scan9FJ5LVCL 3            1           2023-08-01 14:41:06   
sess9FJ5LZ1C scan9FJ5LZ1C 3            1           2023-08-01 14:45:29   
sess9FJ5MGQ9 scan9FJ5MGQ9 3            1           2023-08-01 15:05:15   
sess9FJ5MPW6 scan9FJ5MPW6 3            1           2023-08-01 15:15:11   
sess9FJ5MVC0 scan9FJ5MVC0 3            1           2023-08-01 15:20:38   
sess9FJ1IJYT scan9FJ1IJYT 0            1           2023-08-02 08:33:46   
                                       2           2023-08-02 08:33:46   
sess9FJ1ISOK scan9FJ1ISOK 0            1           2023-08-02 08:35:01   
                                       2           2023-08-02 08:35:01   
             scan9FJ1J0G2 0            1           2023-08-02 08:36:09   
                                       2           2023-08-02 08:36:09   
sess9FJBNGKX scan9FJBNGKX 0            1           2023-08-02 08:37:08   
                                       2           2023-08-02 08:37:08   
sess9FJ66OT2 scan9FJ66OT2 3            1           2023-08-02 09:12:45   
sess9FGLEFJ3 scan9FGLEFJ3 3            1           2023-08-02 09:59:38   
sess9FI3KWP3 scan9FI3KWP3 0            1           2023-08-02 11:50:32   
             scan9FI3L5PZ 0            1           2023-08-02 11:51:29   
sess9FHDA7AI scan9FHDA7AI 0            1           2023-08-02 11:56:57   
             scan9FHDADB9 0            1           2023-08-02 11:57:53   
sess9FGLE1FN scan9FGLE1FN 3            1           2023-08-02 12:29:02   
sess9FGLEFJ3 scan9FGLEFJ3 0            1           2023-08-02 12:40:10   
                          3            2           2023-08-02 12:40:10   
sess9FI8ALGO scan9FI8ALGO 0            1           2023-08-02 14:11:59   
sess9FHS7Y22 scan9FHS7Y22 0            1           2023-08-02 18:01:31   
             scan9FHS845A 0            1           2023-08-02 18:02:33   

                                                                                  curation_output_dir   
session_id   scan_id      paramset_idx curation_id                                                      
sess9FJBPNSP scan9FJBPNSP 0            1            /datajoint-data/data/tobiasr/JJ_WEZ-8873_2023-...  \
                                       2            /datajoint-data/data/tobiasr/JJ_WEZ-8873_2023-...   
                                       3            /datajoint-data/data/tobiasr/JJ_WEZ-8873_2023-...   
sess9FH37WBZ scan9FH37WBZ 0            1            /datajoint-data/data/tobiasr/JJ_ROS-1487_2023-...   
sess9FHELAYA scan9FHEL

## Run imaging processing tasks

In [ ]:
session.Session() * scan.Scan()

In [ ]:
imaging.Curation()

In [ ]:
populate_settings = {'display_progress': True, 'suppress_errors': False, 'processes': 1}
imaging.Processing.populate(**populate_settings)

### Do manual suite2p curation now!
### Then: populate curation table

In [ ]:
imaging.Processing()

In [ ]:
ibe.ingest_aux('sess9FHELAYA', 'scan9FHELAYA')

In [ ]:
imaging.Curation().create1_from_processing_task({'session_id': "sess9FHELAYA", 'scan_id': "scan9FHELAYA", "paramset_idx":1})

Populate imaging.MotionCorrection

In [ ]:
populate_settings = {'display_progress': True, 'suppress_errors': False, 'processes': 60}

In [ ]:
imaging.MotionCorrection.populate(**populate_settings)

In [ ]:
imaging.MotionCorrection.NonRigidMotionCorrection()

In [ ]:
imaging.MotionCorrection.NonRigidMotionCorrection()

In [ ]:
imaging.Segmentation.populate(**populate_settings)

In [ ]:
imaging.MaskClassification.populate(**populate_settings)

In [ ]:
imaging.Fluorescence.populate(**populate_settings)

In [ ]:
imaging.Activity.populate(**populate_settings)

## Exploring data

In [ ]:
# defining queries

sessi = "sess9FHELAYA"
scansi = "scan9FHELAYA"

session_key = (session.Session & f'session_id = "{sessi}"').fetch('KEY')
scan_key = (scan.Scan & f'scan_id = "{scansi}"').fetch('KEY')
curation_key = (imaging.Curation & scan_key & 'curation_id=1').fetch1('KEY')

### Some images / plots

In [ ]:
# TODO: make helper function

# Figure Style settings for notebook.

import matplotlib as mpl
mpl.rcParams.update({
    'axes.spines.left': False,
    'axes.spines.bottom': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'legend.frameon': False,
    'figure.subplot.wspace': .01,
    'figure.subplot.hspace': .01,
    'figure.figsize': (18, 13),
    'ytick.major.left': False,
    'xtick.major.bottom': False
})
jet = mpl.cm.get_cmap('jet')
jet.set_bad(color='k')

In [ ]:

ref_image = (imaging.MotionCorrection.Summary & curation_key & 'field_idx=0').fetch1('ref_image')
average_image = (imaging.MotionCorrection.Summary & curation_key & 'field_idx=0').fetch1('average_image')
correlation_image = (imaging.MotionCorrection.Summary & curation_key & 'field_idx=0').fetch1('correlation_image')
max_proj_image = (imaging.MotionCorrection.Summary & curation_key & 'field_idx=0').fetch1('max_proj_image')

In [ ]:
tmpdir = dj.config['custom'].get('suite2p_tif_tmp')[0]
session_id = curation_key['session_id']

In [ ]:
# TODO: prevent showing and just save. Make function

plt.ion()
plt.subplot(1, 4, 1)
plt.imshow(ref_image, cmap='gray', )
plt.title("Reference Image for Registration");

plt.subplot(1, 4, 2)
plt.imshow(average_image, cmap='gray')
plt.title("Registered Image, Mean Projection");

plt.subplot(1, 4, 3)
plt.imshow(max_proj_image, cmap='gray')
plt.title("Registered Image, Max Projection")

plt.subplot(1, 4, 4)
plt.imshow(correlation_image, cmap='gray')
plt.title("Registered Image, Correlation Map")
plt.show(block=False)
# plt.savefig

In [ ]:
# TODO: prevent showing and just save. Make function
plt.ion()
mask_xpix, mask_ypix = (imaging.Segmentation.Mask * imaging.MaskClassification.MaskType & curation_key & 'mask_center_z=0').fetch('mask_xpix','mask_ypix')

mask_image = np.zeros(np.shape(average_image), dtype=bool)
for xpix, ypix in zip(mask_xpix, mask_ypix):
    mask_image[ypix, xpix] = True

plt.imshow(average_image);
plt.contour(mask_image, colors='white', linewidths=0.5);

In [ ]:
# TODO: prevent showing and just save. Make function
query_cells = (imaging.Segmentation.Mask * imaging.MaskClassification.MaskType & curation_key & 'mask_center_z=0' & 'mask_npix > 1').proj()

fluorescence_traces = (imaging.Fluorescence.Trace & query_cells).fetch('fluorescence', order_by='mask')

activity_traces = (imaging.Activity.Trace & query_cells).fetch('activity_trace', order_by='mask')

sampling_rate = (scan.ScanInfo & curation_key).fetch1('fps') # [Hz]

fig, ax = plt.subplots(1, 1, figsize=(16, 4))
ax2 = ax.twinx()

for f, a in zip(fluorescence_traces, activity_traces):
    ax.plot(np.r_[:f.size] * 1/sampling_rate, f, 'w', label='fluorescence trace')    
    ax2.plot(np.r_[:a.size] * 1/sampling_rate, a, 'r', alpha=0.5, label='deconvolved trace')
    
    break

ax.tick_params(labelsize=14)
ax2.tick_params(labelsize=14)

ax.legend(loc='upper left', prop={'size': 14})
ax2.legend(loc='upper right', prop={'size': 14})

ax.set_xlabel('Time (s)')
ax.set_ylabel('Activity (a.u.)')
ax2.set_ylabel('Activity (a.u.)');

In [ ]:
imaging.Activity.Trace() & query_cells

In [ ]:
imaging.Fluorescence.Trace() * imaging.Segmentation.Mask() & query_cells

In [ ]:
imaging.Segmentation.Mask()

In [ ]:
from rastermap import Rastermap
from scipy import stats 
from scipy.ndimage import gaussian_filter1d

ops = {'n_components': 1, 'n_X': 100, 'alpha': 1., 'K': 1.,
            'nPC': 200, 'constraints': 2, 'annealing': True, 'init': 'pca',
            'start_time': 0, 'end_time': -1}

S = np.vstack(fluorescence_traces)# deconvolved
# S = nXp.vstack(fluorescence_traces)
# we run rastermap the same way that the other scikit-learn embedding algorithms work
# model = Rastermap(n_components=1, n_X=100, verbose = True).fit(S) 
model = Rastermap(n_components=ops['n_components'], n_X=ops['n_X'], nPC=ops['nPC'],
                          init=ops['init'], alpha=ops['alpha'], K=ops['K'], constraints=ops['constraints'],
                          annealing=ops['annealing'])
model.fit(S)
# def running_average(X, nbin = 10):
#     Y = np.cumsum(X, axis=0)
#     Y = Y[nbin:, :] - Y[:-nbin, :]
#     return Y
# the manifold embedding is in model.embedding
isort = np.argsort(model.embedding[:,0])

# sort by embedding and smooth over neurons
# Sfilt = running_average(S[isort], 0)
Sfilt = gaussian_filter1d(S[isort], np.minimum(8,np.maximum(1,int(S.shape[0]*0.001))),axis=0)
# Sfilt = np.vstack(fluorescence_traces)
Sfilt = stats.zscore(Sfilt, axis=1)



In [ ]:
np.shape(S)

In [ ]:
# TODO: make helper function

# Figure Style settings for notebook.

import matplotlib as mpl
mpl.rcParams.update({
    'axes.spines.left': False,
    'axes.spines.bottom': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'legend.frameon': False,
    'figure.subplot.wspace': .01,
    'figure.subplot.hspace': .01,
    'figure.figsize': (18, 13),
    'ytick.major.left': True,
    'xtick.major.bottom': True
})
jet = mpl.cm.get_cmap('jet')
jet.set_bad(color='k')

In [ ]:
plt.figure(figsize=(40,5))
plt.imshow(Sfilt, vmin = -0.1, vmax=4, aspect='auto', cmap='gray_r')
plt.xlabel('time points')
plt.ylabel('sorted neurons')
# plt.show()

also see https://github.com/trose-neuro/mini2p---moser-analysis-datajoint/blob/master/Analysis%20notebooks/PCA%20Stringer%20Pacchitariu.ipynb

In [ ]:
from adamacs.helpers import stack_helpers as sh

In [ ]:
from scipy.ndimage import mean
import tifffile
path = (scan.ScanPath & curation_key).fetch1("path") + ("/suite2p/plane0/reg_tif")

# path = '/datajoint-data/data/jisooj/RN_OPI-1681_2023-02-15_scan9FGLEFJ3_sess9FGLEFJ3/suite2p_exp9FGLEFJ3/suite2p/plane0/reg_tif'
# Get a list of all tiff files in the folder
tiff_files = [os.path.join(path, f) for f in natsorted(os.listdir(path)) if f.endswith('.tif')]

print(tiff_files)

# Load each tiff stack into a list of numpy arrays
stacks = []
for f in tiff_files:
    with tifffile.TiffFile(f) as tif:
        # Get the number of pages in the file
        num_pages = len(tif.pages)
        
        # Create a numpy array to store all pages
        stack = np.zeros((num_pages,) + tif.pages[0].shape, dtype=tif.pages[0].dtype)
        
        # Iterate over the pages and store them in the array
        for i, page in enumerate(tif.pages):
            stack[i] = page.asarray()

    stacks.append(stack)

# Concatenate the stacks into a single numpy array along the z-axis
volume = np.concatenate(stacks, axis=0)

# # delete registration tiffd
# for f in tiff_files:
#     os.remove(f) 

### moving average filter

In [ ]:
### moving average filter
# Create a running Z mean projection of the volume

runav = 40
# running_z_projection = uniform_filter_mt(volume, size=(runav,xyrunav,xyrunav))
running_z_projection = sh.rolling_average_filter_mt(volume, runav)

In [ ]:
sh.display_volume_z(running_z_projection,1)

### mp4 movie generaltion

In [ ]:
filename = os.path.join(path, 'registered_movie' + str(runav) + '_frame_runningaverage' + '.mp4')

fps = 240   # frames per second - 120 default
p1 = 1      # percentile scaling low - 1 default
p2 = 99.995  # percentile scaling high - 99.995 default

rescaled_image_8bit = sh.make_stack_movie(running_z_projection, filename, fps, p1, p2)

## Exploring updates

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(session.Session) + dj.Diagram(scan) + dj.Diagram(imaging.Processing) + dj.Diagram(imaging.Curation) + dj.Diagram(imaging.Curation)

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(session.Session) + dj.Diagram(scan) + dj.Diagram(imaging)

In [ ]:
imaging.MotionCorrection.Summary()
